In [1]:
# Import necessary modules from your files
import os
import subprocess
import numpy as np

from tqdm.notebook import tqdm

from searchclient.agent_types.classic import * 

# Import all action classes (used for hardcoding solutions) and actions libraries
from searchclient.domains.hospital.actions import (
    NoOpAction, MoveAction, PushAction, PullAction, AnyAction, DEFAULT_MAPF_ACTION_LIBRARY, DEFAULT_HOSPITAL_ACTION_LIBRARY
)

# Import state, goal description and level classes for the MAvis hospital environment
from searchclient.domains.hospital.state import HospitalState
from searchclient.domains.hospital.goal_description import HospitalGoalDescription
from searchclient.domains.hospital.level import HospitalLevel

# Import the Graph-Search algorithm
from searchclient.search_algorithms.graph_search import graph_search

# Import the different search strategies for both uninformed and informed search
from searchclient.strategies.bfs import FrontierBFS
from searchclient.strategies.dfs import FrontierDFS
from searchclient.strategies.bestfirst import FrontierBestFirst, FrontierGreedy, FrontierAStar

# Import heuristic classes, to be used in informed search methods
from searchclient.domains.hospital.heuristics import (
    HospitalZeroHeuristic, HospitalGoalCountHeuristics, HospitalAdvancedHeuristics
)

# Ensure the environment is set up properly
print("Modules imported successfully.")

Modules imported successfully.



## Level Design

You can use the [Mavis Level Designer](https://sebastianrmason.github.io/mavis-level-designer/) to create and edit levels.
Save the level as a `.lvl` file and upload it in thele `level` folder for experimentation.

In [2]:
# Function to load a level file
def load_level_file_from_path(path):
    with open(path, "r") as f:
        lines = f.readlines()
        lines = list(map(lambda line: line.strip(), lines))
        return lines
  
# Example usage: load_level('path_to_level_file.lvl')
level_path = "levels/MAPF00.lvl"
level_lines = load_level_file_from_path(level_path)
level = HospitalLevel.parse_level_lines(level_lines)

# We can access the initial state of the level using the following code
initial_state = HospitalState(level, level.initial_agent_positions, level.initial_box_positions)

# We can access the goal description of the level using the following code
goal_description = HospitalGoalDescription(level, level.box_goals + level.agent_goals)

print('The initial state of the level is:')
print(initial_state)

print('\nThe goal description of the level is:')
print(goal_description) # which tells us where the level objects (like boxes and agents) should be placed to satisfy the goal
print('\nSo agent zero starts at {} and satisfies the goal at {}'.format(level.initial_agent_positions[0][0], goal_description.agent_goals[0][0]))

The initial state of the level is:
++++++++++
+0       +
+     ++++
+        +
+     ++ +
+      + +
++++++++++

The goal description of the level is:
((5, 8), '0', True)

So agent zero starts at (1, 1) and satisfies the goal at (5, 8)


## Rendering states
Visualizing the states of your environment is an essential step in understanding the intricacies of your problem space. By rendering individual states, you can gain a better grasp of the initial setup, intermediate configurations, and goal states of your system. This approach not only helps debug your environment but also lays a foundation for interpreting plans and strategies more effectively.

Rendering states can be accomplished easily using the `render_state()` function in the PyGame rendering script. This function takes a level and single state as input and visualizes it, allowing you to observe key attributes and spatial layouts. For instance, you can analyze agent positions, obstacles, or other relevant features specific to your problem.

The function will save the rendered state as a `.png` file. Unless specified, the image will be saved as `rendered_state.png`. Below is an example on how to use it where we just save the initial state of the level above. 

In [3]:
from PIL import Image
from renderState import *

# Render some state of the level (here the initial state)
render_state(level_path=level_path, state=initial_state, output_path='give_me_a_name')

# So the state looks like this in .txt format (what the computer uses):
print(initial_state)

# And the initial state looks like this in .png:
# You can either show the image in the notebook or open it in a new window
img = Image.open('give_me_a_name.png')
img.show()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
State rendered and saved to give_me_a_name.png
++++++++++
+0       +
+     ++++
+        +
+     ++ +
+      + +
++++++++++


## Rendering plans
Rendering your environment is a big part of gaining core insights about the theoretical challenges of certain problems. It helps you gain a conceptual understanding of your chosen approach much faster, and can also aid you in chosing your strategy. Below, we show you an easy way to send the plan found by graph search to the PyGame rendering script.

All we have to do is convert the plan returned by graph search to a string using the `convert_plan_to_string()` utility function, and then pass it to the `renderMAvis.py` script. Note that the first execution of the `render_plan()` function may take some time, but it typically runs much faster on subsequent attempts.

In [4]:
def render_plan(level_path, plan, strategy_name, heuristic_name, num_generated, elapsed_time, sol_length):

    str_plan = convert_plan_to_string(plan) #convert the plan to a string

    # this just makes sure that the meta information is displayed correctly in the visualization
    if strategy_name == 'greedy' or strategy_name == 'astar':
        strategy_name_pygame = strategy_name + ' w. ' + heuristic_name
    else:
        strategy_name_pygame = strategy_name
    
    subprocess.run(["python3", 
                    "renderMAvis.py", 
                    "--level", level_path, 
                    "--plan", str_plan, 
                    "--search_strategy", strategy_name_pygame, 
                    "--num_generated", str(num_generated), 
                    "--time_elapsed", str(elapsed_time), 
                    "--sol_length", str(sol_length)])

We can use the `render_plan` function to hardcode a solution to our problem. Since you will later use Graph-Search to automatically find plans, you can use this to familiarize yourself with the client, the actions class and other classes of the MAvis environment.

In [5]:
# Here is an example of a plan. A plan is a list of list of actions, where an action is an instance from searchclient.domains.hospital.actions
# The first axis of the list corresponds to a timestep in the plan. 
# Each element of the innermost list corresponds to an action of a different agent. 
# Check the documentation of graph_seach.py for more information!

# Hardcode your solution here!
#hardcoded_plan = None
hardcoded_plan = [[MoveAction("S")], 
                  [MoveAction("S")], [MoveAction("E")], 
                  [MoveAction("E")], [MoveAction("E")], 
                  [MoveAction("E")], [MoveAction("E")], 
                  [MoveAction("E")], [MoveAction("E")],
                  [MoveAction("S")], [MoveAction("S")],
                  [MoveAction("S")],  [NoOpAction()]]

render_plan(level_path, hardcoded_plan, "Hardcoded sol", None, len(hardcoded_plan), "0.0", len(hardcoded_plan))

## Executing Search Algorithms

In the following code we will be running the Graph-Search algorithm using a specified search strategy. 

You will have to implement the Graph-Search yourself after what you have seen in class. The implementation for the Bredth First Search (BFS) frontier is already available and you can use it as inspiration for coding other search strategies in the future.

Until Graph-Search is implemented, you are only able to hardcode a plan. Later on you can use the following pipeline to run experiments for the Mavis assignments. 


In [17]:
# Before running the search algorithm, we need to define the action set and the action library
# Use this library for pure pathfinding problems
# action_library = DEFAULT_MAPF_ACTION_LIBRARY

# Use this library for sokoban-like problems (includes Push and Pull actions)
action_library = DEFAULT_HOSPITAL_ACTION_LIBRARY

# Every agent will have the same action set
action_set = [action_library] * level.num_agents

# In order to run Graph-Search, we need to specify the initial state, action set, goal description, and frontier 

# If needed, we need to specify and fetch a heuristic function before initializing the frontier (informed search) 
# When adding new heuristics, remember to update the dictionary! 
# Use a string that matches the name of the heuristic function in the heuristics.py file
heuristic_name = "goalcount" 
heuristic = {
        'zero': HospitalZeroHeuristic,
        'goalcount': HospitalGoalCountHeuristics,
        'advanced': HospitalAdvancedHeuristics,
    }.get(heuristic_name, HospitalZeroHeuristic)() # make sure you understand what .get() does


# Finally, let's pick the search strategy and fetch the relevant frontier
strategy_name = "astar" 
frontier = {
        'bfs': FrontierBFS,
        'dfs': FrontierDFS,
        'astar': lambda: FrontierAStar(heuristic),
        'greedy': lambda: FrontierGreedy(heuristic)
    }.get(strategy_name, FrontierBFS)() # make sure you understand what .get() does

In [18]:
# Now that we have defined the initial state, action set, goal description, and frontier, we can run the search algorithm
planning_success, plan, num_generated, elapsed_time = graph_search(initial_state, action_set, goal_description, frontier)

# The graph search function returns the following:
print('Planning successful:', planning_success)
print('Plan:', plan)
print('Solution length:', len(plan))
print('Number of states generated:', num_generated)
print('Elapsed time:', elapsed_time)

Planning successful: True
Plan: [[Push(E,E)], [Push(E,E)], [Push(E,E)], [Push(E,E)], [Push(E,E)], [Push(E,E)]]
Solution length: 6
Number of states generated: 8.0
Elapsed time: 0.001


#Expanded:        6, #Frontier:        2, #Generated:        8, Time: 0,001 s, Memory: 169,52 MB




In [8]:
render_plan(level_path, plan, strategy_name, heuristic, num_generated, elapsed_time, len(plan))

## Running multiple trials and statistical analysis of performance

In order to make more principled statements over the different algorithms' performance, we will provide you a pipeline to run $n$-trials over a given level. This will allow you to consider the average performance and their statistics instead of drawing conclusions from a singular example. 

**NOTE:** for complex levels with longer runtimes, it might not be feasible to run a high number of trials! Take this into consideration when running your experiments

In [26]:
# For easiness of use, let's redifine and reload everything  
level_path = "levels/MAPF00.lvl" # changed level path to a sokoban map
level_lines = load_level_file_from_path(level_path)
level = HospitalLevel.parse_level_lines(level_lines)

# We can access the initial state of the level using the following code
initial_state = HospitalState(level, level.initial_agent_positions, level.initial_box_positions)

# We can access the goal description of the level using the following code
goal_description = HospitalGoalDescription(level, level.box_goals + level.agent_goals)

print('The initial state of the level is:')
print(initial_state)

print('\nThe goal description of the level is:')
print(goal_description) # which tells us where the level objects (like boxes and agents) should be placed to satisfy the goal
print('\nSo agent zero starts at {} and satisfies the goal at {}'.format(level.initial_agent_positions[0][0], goal_description.agent_goals[0][0]))
# commented out above for sokoban problems

The initial state of the level is:
++++++++++
+0       +
+     ++++
+        +
+     ++ +
+      + +
++++++++++

The goal description of the level is:
((5, 8), '0', True)

So agent zero starts at (1, 1) and satisfies the goal at (5, 8)


In [27]:
# Before running the search algorithm, we need to define the action set and the action library
# Use this library for pure pathfinding problems
action_library = DEFAULT_MAPF_ACTION_LIBRARY

# Use this library for sokoban-like problems (includes Push and Pull actions)
# action_library = DEFAULT_HOSPITAL_ACTION_LIBRARY

# Every agent will have the same action set
action_set = [action_library] * level.num_agents

# In order to run Graph-Search, we need to specify the initial state, action set, goal description, and frontier 

# If needed, we need to specify and fetch a heuristic function before initializing the frontier (informed search) 
# When adding new heuristics, remember to update the dictionary! 
# Use a string that matches the name of the heuristic function in the heuristics.py file
heuristic_name = "goalcount" 
heuristic = {
        'zero': HospitalZeroHeuristic,
        'goalcount': HospitalGoalCountHeuristics,
        'advanced': HospitalAdvancedHeuristics,
    }.get(heuristic_name, HospitalZeroHeuristic)() # make sure you understand what .get() does


# Finally, let's pick the search strategy and fetch the relevant frontier
strategy_name = "astar" 
frontier = {
        'bfs': FrontierBFS,
        'dfs': FrontierDFS,
        'astar': lambda: FrontierAStar(heuristic),
        'greedy': lambda: FrontierGreedy(heuristic)
    }.get(strategy_name, FrontierBFS)() # make sure you understand what .get() does

In [29]:
n_trials = 10

plans, sol_lengths, generated, elapsed = [], [], [], []
for n in tqdm(range(n_trials)):
    planning_success, plan, num_generated, elapsed_time = graph_search(initial_state, action_set, goal_description, frontier)
    plans.append(plan)
    sol_lengths.append(len(plan))
    generated.append(int(num_generated))
    elapsed.append(elapsed_time)


# The graph search function returns the following:
print('Average solution legth:', np.mean(sol_lengths))
print('Solution length variance :', np.var(sol_lengths))
print('Average number of states generated:', np.mean(generated))
print('Number of states generated variance :', np.var(generated))

  0%|          | 0/10 [00:00<?, ?it/s]

Average solution legth: 11.0
Solution length variance : 0.0
Average number of states generated: 11.0
Number of states generated variance : 0.0


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,002 s, Memory: 170,36 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,001 s, Memory: 170,36 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,001 s, Memory: 170,36 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,000 s, Memory: 170,36 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,001 s, Memory: 170,36 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,001 s, Memory: 170,36 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,000 s, Memory: 170,36 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,002 s, Memory: 170,37 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,000 s, Memory: 170,37 MB


#Expanded:       11, #Frontier:        0, #Generated:       11, Time: 0,002 s, Memory: 170,37 MB




In [ ]:
# In case you want to visualize the i-th trial

# Select trial to render
index_to_render = 0
# Render trial
render_plan(
    level_path,
    plans[index_to_render], 
    strategy_name, 
    heuristic, 
    generated[index_to_render],
    elapsed[index_to_render], 
    sol_lengths[index_to_render]
)